# Big Trade Monitor Demo

This notebook demonstrates the Big Trade Monitor that automatically detects and alerts on large trades across all Polymarket orderbooks.

In [ ]:
# Import required modules
import sys
sys.path.append('.')

from polymarket_orderbook.big_trade_monitor import BigTradeMonitor, print_alert, log_alert
from polymarket_orderbook.orderbook_client import OrderbookClient
import time

## 1. Create and Configure Monitor

In [ ]:
# Create the monitor
monitor = BigTradeMonitor(
    poll_interval=5.0,        # Check every 5 seconds
    size_threshold=100.0,     # Alert on trades >= 100 size
    value_threshold=50.0      # Alert on trades >= $50 value
)

print("✅ Monitor created successfully")

## 2. Add Alert Handlers

In [ ]:
# Add print alert (displays in console)
monitor.add_alert_callback(print_alert)

# Add log alert (saves to file)
monitor.add_alert_callback(lambda trade: log_alert(trade, "big_trades.log"))

# Add custom alert
def custom_alert(trade):
    """Custom alert handler example"""
    value = trade.get('value', 0)
    if value > 500:
        print(f"💰 HUGE TRADE: ${value:,.2f}")

monitor.add_alert_callback(custom_alert)

print("✅ Alert handlers registered")

## 3. Discover Markets to Monitor

In [ ]:
# Discover active markets
markets = monitor.discover_all_active_markets(limit=30)

print(f"\nDiscovered {len(markets)} markets")
print("\nSample markets:")
for i, market in enumerate(markets[:5]):
    print(f"{i+1}. {market.get('question', 'N/A')[:70]}")
    print(f"   Outcome: {market.get('outcome', 'N/A')}")
    print(f"   Token ID: {market.get('token_id', 'N/A')[:20]}...")
    print()

## 4. Start Monitoring

In [ ]:
# Set markets to monitor
monitor.set_tracked_markets(markets)

# Start the monitor (runs in background thread)
monitor.start(auto_discover=False)

print("\n✅ Monitor is now running in the background!")
print("It will automatically alert when big trades are detected.")
print("\nLet it run for a bit to see alerts...")

## 5. Monitor Stats (Check After Running)

In [ ]:
# Wait a bit for some data
print("Monitoring for 60 seconds...")
time.sleep(60)

# Print statistics
monitor.print_stats()

## 6. View Detected Big Trades

In [ ]:
# Get big trades as DataFrame
big_trades_df = monitor.get_big_trades_dataframe()

if len(big_trades_df) > 0:
    print(f"\n📊 Detected {len(big_trades_df)} big trades:\n")
    print(big_trades_df.select(['timestamp', 'side', 'size', 'price', 'value', 'outcome']))
else:
    print("\n⏳ No big trades detected yet. Let the monitor run longer...")

## 7. Adjust Thresholds (Optional)

In [ ]:
# Make it more sensitive (lower thresholds)
monitor.set_thresholds(size_threshold=50.0, value_threshold=25.0)

print("✅ Thresholds updated - monitor will now be more sensitive")
print("\nLet it run for another minute...")
time.sleep(60)

monitor.print_stats()

## 8. Extended Monitoring (Run for 3 minutes)

In [ ]:
print("Running extended monitoring for 3 minutes...")
print("Watch for big trade alerts!\n")

for i in range(3):
    print(f"Minute {i+1}/3...")
    time.sleep(60)
    
    # Show current stats
    stats = monitor.get_stats()
    print(f"  ✓ Trades checked: {stats['total_trades_checked']}")
    print(f"  ✓ Big trades: {stats['big_trades_detected']}")
    print()

print("\n" + "="*60)
monitor.print_stats()

## 9. View All Big Trades Summary

In [ ]:
# Get all big trades
big_trades_df = monitor.get_big_trades_dataframe()

if len(big_trades_df) > 0:
    print(f"\n📊 SUMMARY: {len(big_trades_df)} Big Trades Detected\n")
    
    # Show detailed view
    print("\nDetailed Big Trades:")
    display_cols = ['timestamp', 'side', 'size', 'price', 'value', 'outcome', 'question']
    available_cols = [col for col in display_cols if col in big_trades_df.columns]
    print(big_trades_df.select(available_cols))
    
    # Statistics
    print("\n📈 Trade Statistics:")
    if 'value' in big_trades_df.columns:
        total_value = big_trades_df['value'].sum()
        avg_value = big_trades_df['value'].mean()
        max_value = big_trades_df['value'].max()
        print(f"Total Value: ${total_value:,.2f}")
        print(f"Average Value: ${avg_value:,.2f}")
        print(f"Max Value: ${max_value:,.2f}")
    
    if 'side' in big_trades_df.columns:
        print("\n📊 By Side:")
        print(big_trades_df.group_by('side').agg(pl.count()))
else:
    print("\n⏳ No big trades detected yet.")
    print("Try:")
    print("  1. Lowering thresholds (set_thresholds)")
    print("  2. Running monitor longer")
    print("  3. Adding more markets")

## 10. Stop Monitor

In [ ]:
# Stop the monitor
monitor.stop()

# Final stats
print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
monitor.print_stats()

# Check log file
import os
if os.path.exists("big_trades.log"):
    print("\n📄 Log file created: big_trades.log")
    with open("big_trades.log", "r") as f:
        lines = f.readlines()
        print(f"Total log entries: {len(lines)}")
        if lines:
            print("\nLast 3 entries:")
            for line in lines[-3:]:
                print("  " + line.strip())

## Test Complete! 🎉

The Big Trade Monitor:
- ✅ Automatically discovers all active markets
- ✅ Continuously monitors trades across all orderbooks
- ✅ Detects big trades based on configurable thresholds
- ✅ Sends real-time alerts via callbacks
- ✅ Logs trades to file
- ✅ Tracks statistics and history
- ✅ Runs in background thread